# Cross-Lakehouse Copy: Shortcuts to Bronze

**IMPORTANT SETUP FOR CROSS-LAKEHOUSE COPY:**

This notebook copies SalesLT data from shortcuts to a specific bronze lakehouse:
- **Source**: `Gaiye_Test_Lakehouse` (contains shortcuts)  
- **Target**: `RDS_Fabric_Foundry_workspace_Gaiye_Retail_Solution_Test_IDM_LH_bronze`

**Required Configuration in Fabric:**
1. **Default Lakehouse**: Set `RDS_Fabric_Foundry_workspace_Gaiye_Retail_Solution_Test_IDM_LH_bronze` as default
2. **Additional Lakehouse**: Add `Gaiye_Test_Lakehouse` as additional lakehouse
3. **Data Flow**: Shortcuts → Bronze Layer Files/bronze/saleslt/

**Expected Result:** Bronze layer created in target lakehouse with organized SalesLT data structure.

## Step 1 - Overview

In [ ]:
# BRONZE LAKEHOUSE VALIDATION AND SETUP
print("🎯 BRONZE LAKEHOUSE CONFIGURATION CHECK")
print("=" * 60)

# Check current lakehouse context
try:
    # Method 1: Check default lakehouse name
    default_tables = spark.sql("SHOW TABLES").toPandas()
    print(f"✅ Default lakehouse connected with {len(default_tables)} tables")
    
    # Method 2: Check Files directory (should be bronze lakehouse)
    try:
        files_list = dbutils.fs.ls("Files/")
        print(f"✅ Files directory accessible with {len(files_list)} items")
        
        # Check if this looks like the bronze lakehouse
        current_items = [item.name.rstrip('/') for item in files_list]
        print(f"📁 Current lakehouse contains: {current_items}")
        
        # This should be our target bronze lakehouse
        print(f"🎯 Target: Write bronze data to Files/bronze/saleslt/")
        
    except Exception as e:
        print(f"⚠️ Cannot access Files directory: {str(e)}")
        
except Exception as e:
    print(f"❌ Lakehouse check failed: {str(e)}")

# Check available databases/lakehouses
try:
    databases = spark.sql("SHOW DATABASES").toPandas()
    print(f"\n📊 Available databases/lakehouses:")
    for db in databases.iloc[:, 0]:
        print(f"   📂 {db}")
        
    # Look for our source lakehouse (Gaiye_Test_Lakehouse)
    source_candidates = [db for db in databases.iloc[:, 0] if 'gaiye' in db.lower() or 'test' in db.lower()]
    if source_candidates:
        print(f"\n🔍 Potential source lakehouses: {source_candidates}")
    else:
        print(f"\n⚠️ Source lakehouse (Gaiye_Test_Lakehouse) not found in databases")
        print(f"💡 Ensure Gaiye_Test_Lakehouse is attached as additional lakehouse")
        
except Exception as e:
    print(f"❌ Database check failed: {str(e)}")

print("\n" + "=" * 60)
print("💡 SETUP REMINDER:")
print("   1. This notebook should run in: RDS_Fabric_Foundry_workspace_Gaiye_Retail_Solution_Test_IDM_LH_bronze")
print("   2. Gaiye_Test_Lakehouse should be attached as additional lakehouse")
print("   3. Data will be written to Files/bronze/saleslt/ in the current (bronze) lakehouse")
print("=" * 60)

## Step 2 - Bronze Lakehouse Validation

## Step 3 - Cross-Lakehouse Configuration

In [ ]:
# CROSS-LAKEHOUSE COPY CONFIGURATION
print("🔄 CONFIGURING CROSS-LAKEHOUSE COPY")
print("=" * 60)

# Configuration for cross-lakehouse copy
TARGET_BRONZE_LAKEHOUSE = "RDS_Fabric_Foundry_workspace_Gaiye_Retail_Solution_Test_IDM_LH_bronze"
SOURCE_SHORTCUTS_LAKEHOUSE = "Gaiye_Test_Lakehouse"

print(f"📤 Source: {SOURCE_SHORTCUTS_LAKEHOUSE} (shortcuts)")
print(f"📥 Target: {TARGET_BRONZE_LAKEHOUSE} (bronze layer)")

# CRITICAL: Determine the correct path for target lakehouse
# In Fabric, when multiple lakehouses are attached, we need to route correctly
print(f"\n🎯 LAKEHOUSE ROUTING CONFIGURATION:")

# Check available lakehouse mounts
try:
    # List available mount points
    mount_points = dbutils.fs.mounts()
    print(f"📁 Available mount points:")
    
    target_mount = None
    for mount in mount_points:
        mount_point = mount.mountPoint
        source_info = mount.source
        print(f"   🔗 {mount_point} → {source_info}")
        
        # Look for the target bronze lakehouse
        if TARGET_BRONZE_LAKEHOUSE.lower() in source_info.lower():
            target_mount = mount_point
            print(f"   ✅ Found target lakehouse mount: {target_mount}")
    
    if target_mount:
        # Use specific mount point for bronze lakehouse
        BRONZE_BASE_PATH = f"{target_mount}bronze/saleslt"
        print(f"🎯 Bronze data will be written to: {BRONZE_BASE_PATH}")
    else:
        # Fallback to default Files/ (current lakehouse)
        BRONZE_BASE_PATH = "Files/bronze/saleslt"
        print(f"⚠️ Using default Files/ path: {BRONZE_BASE_PATH}")
        print(f"💡 Ensure {TARGET_BRONZE_LAKEHOUSE} is set as default lakehouse")
        
except Exception as e:
    print(f"❌ Mount check failed: {str(e)}")
    # Use default path
    BRONZE_BASE_PATH = "Files/bronze/saleslt"
    print(f"🔄 Using default path: {BRONZE_BASE_PATH}")

# Alternative method: Use abfss:// path for specific lakehouse
print(f"\n🔍 ALTERNATIVE: Direct lakehouse path")
try:
    # Check if we can determine workspace and lakehouse info
    workspace_info = spark.sql("DESCRIBE DETAIL default.information_schema").collect()
    print(f"📊 Current workspace context available")
    
    # Construct specific lakehouse path
    BRONZE_ABFSS_PATH = f"abfss://bronze@onelake.dfs.fabric.microsoft.com/bronze/saleslt"
    print(f"🔗 Alternative ABFSS path: {BRONZE_ABFSS_PATH}")
    
except Exception as e:
    print(f"⚠️ Workspace info not available: {str(e)}")

# Override connection method for shortcut-based copy
FORCE_SHORTCUT_COPY = True
CONNECTION_METHOD_OVERRIDE = "existing_tables"  # Use existing tables (shortcuts)

print(f"\n📋 FINAL CONFIGURATION:")
print(f"   📤 Source Method: {CONNECTION_METHOD_OVERRIDE}")
print(f"   📥 Target Path: {BRONZE_BASE_PATH}")
print(f"   🎯 Target Lakehouse: {TARGET_BRONZE_LAKEHOUSE}")

# Verify we can write to the target location
print(f"\n🧪 TESTING WRITE ACCESS:")
try:
    # Test write access to target path
    test_path = f"{BRONZE_BASE_PATH}/_connection_test/"
    
    # Create test dataframe
    test_df = spark.createDataFrame([{"test": "connection", "timestamp": str(datetime.now())}])
    
    # Test write
    test_df.write.mode("overwrite").parquet(test_path)
    
    # Verify write
    test_read = spark.read.parquet(test_path)
    test_count = test_read.count()
    
    # Clean up test
    dbutils.fs.rm(test_path, True)
    
    print(f"✅ Write access confirmed to {BRONZE_BASE_PATH}")
    print(f"✅ Test file created and read successfully ({test_count} records)")
    print(f"✅ Ready to copy to bronze layer in target lakehouse")
    
except Exception as e:
    print(f"❌ Write access test failed: {str(e)}")
    print(f"💡 TROUBLESHOOTING:")
    print(f"   1. Ensure this notebook is attached to: {TARGET_BRONZE_LAKEHOUSE}")
    print(f"   2. Or ensure {TARGET_BRONZE_LAKEHOUSE} is set as default lakehouse")
    print(f"   3. Check lakehouse permissions and access")

# Check for source data (shortcuts)
print(f"\n🔍 CHECKING SOURCE SHORTCUTS:")
try:
    # Look for SalesLT-related tables
    all_tables = spark.sql("SHOW TABLES").toPandas()
    
    # Expected shortcut table names (without SalesLT prefix)
    expected_tables = ['address', 'customer', 'customeraddress', 'product', 
                      'productcategory', 'productdescription', 'productmodel',
                      'productmodelproductdescription', 'salesorderdetail', 'salesorderheader']
    
    found_shortcuts = []
    for expected in expected_tables:
        matches = all_tables[all_tables['tableName'].str.lower() == expected.lower()]
        if not matches.empty:
            found_shortcuts.append(expected)
            print(f"   ✅ Found shortcut: {expected}")
    
    if len(found_shortcuts) > 0:
        print(f"\n🎉 Found {len(found_shortcuts)} shortcuts ready for copy")
        print(f"📋 Will copy: {', '.join(found_shortcuts)}")
        
        # Store configuration for export function
        globals()['BRONZE_TARGET_PATH'] = BRONZE_BASE_PATH
        globals()['SOURCE_TABLES'] = found_shortcuts
        
    else:
        print(f"\n⚠️ No shortcuts found in accessible tables")
        print(f"💡 Available tables: {', '.join(all_tables['tableName'].tolist()[:10])}...")
        print(f"💡 Ensure {SOURCE_SHORTCUTS_LAKEHOUSE} is attached as additional lakehouse")

except Exception as e:
    print(f"❌ Shortcut check failed: {str(e)}")

print("\n" + "=" * 60)
print("🚀 Configuration complete - ready for cross-lakehouse copy!")
print(f"🎯 Data will be written to: {BRONZE_BASE_PATH}")
print("=" * 60)

## 🎯 CROSS-LAKEHOUSE COPY SETUP GUIDE

To copy SalesLT data from shortcuts to your bronze lakehouse, follow these steps in Microsoft Fabric:

### 1. **Open Target Lakehouse** 
- Navigate to `RDS_Fabric_Foundry_workspace_Gaiye_Retail_Solution_Test_IDM_LH_bronze`
- This will be your **default lakehouse** (where data gets written)

### 2. **Import This Notebook**
- Import this notebook into the target lakehouse workspace
- The notebook will write data to `Files/bronze/saleslt/` in the current lakehouse

### 3. **Attach Source Lakehouse**
- In the notebook, click **"Add Lakehouse"** 
- Select `Gaiye_Test_Lakehouse` as **additional lakehouse**
- This gives access to the shortcuts for reading data

### 4. **Run the Notebook**
- Execute the configuration cells (cells 1-4) to verify setup
- Run the export process to copy all SalesLT tables
- Data will appear in `Files/bronze/saleslt/` in your bronze lakehouse

### 5. **Verify Results**
- Check `Files → bronze → saleslt` in the bronze lakehouse
- You should see folders for each table (customer, product, etc.)
- Each folder contains parquet files with your SalesLT data

### Expected Result:
```
RDS_Fabric_Foundry_workspace_Gaiye_Retail_Solution_Test_IDM_LH_bronze/
└── Files/
    └── bronze/
        └── saleslt/
            ├── address/
            ├── customer/
            ├── customeraddress/
            ├── product/
            ├── productcategory/
            ├── productdescription/
            ├── productmodel/
            ├── productmodelproductdescription/
            ├── salesorderdetail/
            └── salesorderheader/
```

**⚡ The key is running this notebook FROM the bronze lakehouse with shortcuts attached!**

## Step 4 - Setup Guide

# Export SalesLT Tables to Retail Data Model Bronze Layer

This notebook dynamically discovers and exports all tables from the SalesLT schema in Gaiye-SQL-DB to the retail data model bronze layer.

**Prerequisites:**
- Fabric workspace with access to Gaiye-SQL-DB
- Retail data model lakehouse attached to this notebook
- Appropriate permissions for SQL database and lakehouse access

In [ ]:
## Step 1: Import Required Libraries

# Import required libraries (Fabric-compatible only)
import pandas as pd
from datetime import datetime
import os
import logging
import json

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("✅ Libraries imported successfully")
print(f"📅 Export started at: {datetime.now()}")
print("🔧 Using Fabric-native connectivity (no external dependencies)")

## Step 5 - Import Libraries

In [ ]:
## Step 2: Environment Diagnostic & Connectivity Check

# This cell validates the Fabric environment and ensures all prerequisites are met
# Key validations:
# ✅ Spark session is active and functional
# ✅ Database connectivity (lists all available databases in workspace)
# ✅ Lakehouse is properly attached to this notebook
# ✅ File system access for bronze layer creation
# ⚠️ Troubleshooting: Fixes common column name issues and auto-creates directories

# Check if we're running in Fabric environment first
try:
    spark_version = spark.version
    print(f"🏭 Fabric Spark session detected: {spark_version}")
except NameError:
    print("⚠️  Spark session not available - ensure you're running in Fabric")
    print("❌ Cannot proceed without Spark session")
    raise Exception("Spark session required for this notebook")

# Fabric Environment Diagnostic Check
print("\n🔍 FABRIC ENVIRONMENT DIAGNOSTIC")
print("=" * 50)

# Check 1: Spark session
try:
    print(f"✅ Spark session active: {spark.version}")
except:
    print("❌ Spark session not available")

# Check 2: Available databases
try:
    databases = spark.sql("SHOW DATABASES").toPandas()
    print(f"✅ Available databases ({len(databases)}):")
    # Handle different possible column names
    db_column = 'namespace' if 'namespace' in databases.columns else 'databaseName'
    for db in databases[db_column]:
        print(f"   📁 {db}")
except Exception as e:
    print(f"❌ Cannot list databases: {str(e)}")

# Check 3: Lakehouse attachment
try:
    tables = spark.sql("SHOW TABLES").toPandas()
    print(f"✅ Lakehouse tables visible ({len(tables)}):")
    for table in tables['tableName'][:5]:  # Show first 5
        print(f"   📊 {table}")
    if len(tables) > 5:
        print(f"   ... and {len(tables)-5} more")
except Exception as e:
    print(f"❌ Cannot access lakehouse tables: {str(e)}")

# Check 4: File system access
try:
    files_path = "Files"
    if os.path.exists(files_path):
        print(f"✅ File system access: {files_path} exists")
    else:
        print(f"⚠️  File system: {files_path} not found - will be created during export")
        # Create the Files directory
        os.makedirs(files_path, exist_ok=True)
        print(f"✅ Created {files_path} directory")
except Exception as e:
    print(f"❌ File system access failed: {str(e)}")

# Check 5: SQL database access (will be checked after database name is defined)
print("⏳ SQL Database access check will run after database configuration...")

print("\n📋 Diagnostic complete. Address any ❌ issues before proceeding.")

## Step 6 - Environment Diagnostic

In [ ]:
## Step 3: Azure SQL Authentication & Connection Methods

# Based on Fabric community guidance, we'll try multiple authentication approaches
# Reference: https://community.fabric.microsoft.com/t5/Data-Engineering/Connecting-to-Azure-SQL-Server-from-a-Fabric-Compute-Notebook/m-p/4414950

print("? AZURE SQL AUTHENTICATION & CONNECTION SETUP")
print("=" * 60)
print("✅ Found Fabric connections (for Data Factory):")
print("   1. FabricSql gazho (Fabric SQL database)")
print("   2. gaiye-sql-db.sql.fabric.microsoft;SalesLT gazho (SQL Server)")
print()
print("💡 Testing authentication methods based on community guidance...")

schema_name = "SalesLT"
working_connection = None
working_syntax = None
working_auth_method = None

# Server details for your Gaiye SQL DB
server_name = "gaiye-sql-db.sql.fabric.microsoft.com"
database_name = "Gaiye-SQL-DB"

print("🎯 Testing Fabric SQL authentication methods:")

# Method 1: Check if there are any shortcuts or external tables already set up
print("\n📍 Method 1: Checking for existing shortcuts/external tables")
try:
    existing_tables = spark.sql("SHOW TABLES").toPandas()
    
    # Standard SalesLT table names (shortcuts import without schema prefix)
    expected_saleslt_tables = [
        'address', 'customer', 'customeraddress', 'product', 
        'productcategory', 'productdescription', 'productmodel',
        'productmodelproductdescription', 'salesorderdetail', 'salesorderheader'
    ]
    
    # Look for exact matches first
    saleslt_tables = []
    for _, row in existing_tables.iterrows():
        table_name = row['tableName'].lower()
        if table_name in expected_saleslt_tables:
            saleslt_tables.append(row['tableName'])
    
    # If no exact matches, look for partial matches
    if not saleslt_tables:
        for _, row in existing_tables.iterrows():
            table_name = row['tableName'].lower()
            if any(expected in table_name or table_name in expected for expected in expected_saleslt_tables):
                saleslt_tables.append(row['tableName'])
    
    if saleslt_tables:
        print(f"   ✅ Found {len(saleslt_tables)} existing SalesLT tables in lakehouse:")
        for table in saleslt_tables:
            print(f"      🔗 {table}")
        print("   🎯 These appear to be lakehouse shortcuts - perfect!")
        working_syntax = "existing_tables"
        working_connection = "lakehouse_shortcuts"
        working_auth_method = "lakehouse_native"
    else:
        print("   ❌ No SalesLT tables found in lakehouse shortcuts")
        print("   💡 Expected tables: Address, Customer, CustomerAddress, Product,")
        print("       ProductCategory, ProductDescription, ProductModel, etc.")
        
except Exception as e:
    print(f"   ❌ Error checking existing tables: {str(e)[:100]}...")

# Method 2: Spark JDBC with Workspace Identity (recommended approach from community)
if not working_connection:
    print("\n📍 Method 2: Spark JDBC with Workspace Identity")
    
    try:
        # Build JDBC URL with ActiveDirectoryMSI authentication
        jdbc_url = f"jdbc:sqlserver://{server_name}:1433;database={database_name};encrypt=true;trustServerCertificate=false;Authentication=ActiveDirectoryMSI"
        
        print(f"   🔍 Trying Spark JDBC with MSI: {server_name}")
        print(f"   📝 JDBC URL: {jdbc_url}")
        
        # Test with a simple query to INFORMATION_SCHEMA
        test_query = f"(SELECT COUNT(*) as table_count FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = '{schema_name}' AND TABLE_TYPE = 'BASE TABLE') AS test"
        
        # Use Spark JDBC connector
        df_test = spark.read \
            .format("com.microsoft.sqlserver.jdbc.spark") \
            .option("url", jdbc_url) \
            .option("dbtable", test_query) \
            .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()
        
        # Execute the query
        result = df_test.collect()
        table_count = result[0]['table_count']
        
        print(f"   ✅ SUCCESS! Spark JDBC with MSI connected, found {table_count} tables")
        working_connection = jdbc_url
        working_syntax = "spark_jdbc_msi"
        working_auth_method = "workspace_identity"
        
    except Exception as e:
        print(f"   ❌ Spark JDBC MSI failed: {str(e)[:100]}...")

# Method 3: Try with mssparkutils token (though community reports issues)
if not working_connection:
    print("\n📍 Method 3: Spark JDBC with Token Authentication")
    
    try:
        # Try to get token for Azure SQL
        print("   🔑 Attempting to get Azure SQL token...")
        
        # Get token for Azure SQL Database
        token = mssparkutils.credentials.getToken(audience="https://database.windows.net/")
        print(f"   📋 Token retrieved (length: {len(token) if token else 0})")
        
        # Build JDBC URL without authentication
        jdbc_url_token = f"jdbc:sqlserver://{server_name}:1433;database={database_name};encrypt=true;trustServerCertificate=false"
        
        print(f"   🔍 Trying Spark JDBC with token: {server_name}")
        
        # Test with token - this is tricky with Spark JDBC, let's try a different approach
        # Note: Community reports this often returns user token instead of workspace token
        test_query = f"(SELECT COUNT(*) as table_count FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = '{schema_name}' AND TABLE_TYPE = 'BASE TABLE') AS test"
        
        # Try to use the token in connection properties
        connection_properties = {
            "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver",
            "accessToken": token
        }
        
        df_test = spark.read.jdbc(
            url=jdbc_url_token,
            table=test_query,
            properties=connection_properties
        )
        
        result = df_test.collect()
        table_count = result[0]['table_count']
        
        print(f"   ✅ SUCCESS! Spark JDBC with token connected, found {table_count} tables")
        working_connection = jdbc_url_token
        working_syntax = "spark_jdbc_token"
        working_auth_method = "token_based"
        
    except Exception as e:
        print(f"   ❌ Spark JDBC token failed: {str(e)[:100]}...")

# Method 4: Check for Fabric SQL Database connections
if not working_connection:
    print("\n📍 Method 4: Fabric SQL Database Direct Connection")
    
    fabric_sql_variations = [
        "FabricSql_gazho",
        "FabricSqlgazho", 
        "FabricSql",
        "gaiye_sql_db"
    ]
    
    for conn_name in fabric_sql_variations:
        try:
            test_query = f"""
            SELECT COUNT(*) as table_count 
            FROM {conn_name}.INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_SCHEMA = '{schema_name}' 
            AND TABLE_TYPE = 'BASE TABLE'
            """
            
            print(f"   🔍 Trying Fabric SQL connection: {conn_name}")
            result = spark.sql(test_query).toPandas()
            table_count = result.iloc[0]['table_count']
            
            print(f"   ✅ SUCCESS! Connected via {conn_name}, found {table_count} tables")
            working_connection = conn_name
            working_syntax = "fabric_sql_direct"
            working_auth_method = "fabric_native"
            break
            
        except Exception as e:
            print(f"   ❌ Failed {conn_name}: {str(e)[:80]}...")
            continue

# Method 5: Check for mirrored databases (fallback)
if not working_connection:
    print("\n📍 Method 5: Check for mirrored SQL databases")
    
    try:
        databases = spark.sql("SHOW DATABASES").toPandas()
        db_column = 'namespace' if 'namespace' in databases.columns else 'databaseName'
        
        sql_candidates = []
        for db in databases[db_column]:
            if any(keyword in db.lower() for keyword in ['gaiye', 'sql', 'server', 'saleslt']):
                sql_candidates.append(db)
        
        print(f"   📁 Found potential SQL-related databases: {sql_candidates}")
        
        for candidate_db in sql_candidates:
            try:
                tables_df = spark.sql(f"SHOW TABLES IN {candidate_db}").toPandas()
                
                if 'tableName' in tables_df.columns:
                    saleslt_in_db = tables_df[tables_df['tableName'].str.contains('saleslt|customer|product|address', case=False, na=False)]
                else:
                    col_name = tables_df.columns[0] if len(tables_df.columns) > 0 else 'table'
                    saleslt_in_db = tables_df[tables_df[col_name].str.contains('saleslt|customer|product|address', case=False, na=False)]
                
                if not saleslt_in_db.empty:
                    table_names_in_db = saleslt_in_db.iloc[:, 0].tolist()
                    saleslt_indicators = ['customer', 'product', 'address', 'salesorder']
                    found_indicators = [name for name in table_names_in_db if any(ind in name.lower() for ind in saleslt_indicators)]
                    
                    if len(found_indicators) >= 2:
                        print(f"   🎯 {candidate_db} looks like SalesLT source: {found_indicators}")
                        working_connection = candidate_db
                        working_syntax = "mirrored_database"
                        working_auth_method = "mirrored_native"
                        break
                        
            except Exception as e:
                continue
        
    except Exception as e:
        print(f"   ❌ Error checking mirrored databases: {str(e)[:100]}...")

# Summary
if working_connection:
    print(f"\n🎉 AUTHENTICATION & CONNECTION SUCCESSFUL!")
    print(f"   ✅ Working method: {working_syntax}")
    print(f"   ✅ Connection/Database: {working_connection}")
    print(f"   ✅ Authentication: {working_auth_method}")
    print(f"   ✅ Schema: {schema_name}")
    
    # Store for use in subsequent cells
    connection_name = working_connection
    connection_method = working_syntax
    auth_method = working_auth_method
    
else:
    print(f"\n⚠️  NO AUTHENTICATION METHOD WORKED")
    print("💡 Based on community feedback, this is a known limitation")
    print("📋 Recommended solutions:")
    print("   1. 🔧 Create lakehouse shortcuts to SQL Server tables")
    print("   2. 🔧 Use Data Factory with existing connections")
    print("   3. 🔧 Set up SQL database mirroring in Fabric")
    print("   4. 🔧 Request workspace identity permissions on SQL Server")
    print("   5. 📋 Use fallback with standard SalesLT table list")
    
    # Set defaults for fallback approach
    connection_name = "manual_setup_required" 
    connection_method = "use_data_factory_or_shortcuts"
    auth_method = "manual_authentication_needed"

print(f"\n📋 Configuration for subsequent cells:")
print(f"   Method: {connection_method}")
print(f"   Connection: {connection_name}")
print(f"   Authentication: {auth_method}")
print(f"   Schema: {schema_name}")

# Additional guidance based on authentication results
if working_auth_method == "workspace_identity":
    print(f"\n💡 WORKSPACE IDENTITY SUCCESS:")
    print(f"   Your Fabric workspace has proper SQL Server access")
    print(f"   This is the recommended approach for production use")
elif working_auth_method == "token_based":
    print(f"\n⚠️  TOKEN-BASED CONNECTION:")
    print(f"   Note: Community reports token may be user-based, not workspace-based")
    print(f"   Monitor for any permission issues during export")
elif working_auth_method == "fabric_native":
    print(f"\n💡 FABRIC SQL DATABASE DETECTED:")
    print(f"   Direct connection to Fabric SQL Database established")
elif auth_method == "manual_authentication_needed":
    print(f"\n🔧 AUTHENTICATION SETUP REQUIRED:")
    print(f"   1. Option A: Grant workspace identity access to SQL Server")
    print(f"      - In Azure portal, go to your SQL Server")
    print(f"      - Add your Fabric workspace as SQL admin or db_datareader")
    print(f"   2. Option B: Create lakehouse shortcuts")
    print(f"      - Go to your lakehouse")
    print(f"      - Create shortcuts to SQL Server tables")
    print(f"   3. Option C: Use Data Factory")
    print(f"      - Your existing connections work in Data Factory")
    print(f"      - Create copy activities for each table")

print("=" * 60)

## Step 7 - Authentication Methods

In [ ]:
## Step 4: Discover SalesLT Tables with Authentication-Aware Methods

print("🔍 DISCOVERING SALESLT TABLES")
print("=" * 60)
print(f"📋 Using authentication method: {auth_method}")
print(f"? Connection: {connection_name}")
print(f"? Method: {connection_method}")
print()

tables_info = []
schema_name = "SalesLT"

# Discovery method based on successful authentication
if connection_method == "spark_jdbc_msi":
    print("💡 Using Spark JDBC with Workspace Identity")
    try:
        # Query INFORMATION_SCHEMA using established JDBC connection
        query = f"""
        (SELECT 
            TABLE_NAME,
            TABLE_TYPE,
            CASE 
                WHEN TABLE_NAME LIKE '%Customer%' THEN 'Customer Data'
                WHEN TABLE_NAME LIKE '%Product%' THEN 'Product Catalog'
                WHEN TABLE_NAME LIKE '%SalesOrder%' THEN 'Sales Transactions'
                WHEN TABLE_NAME LIKE '%Address%' THEN 'Address Information'
                ELSE 'Reference Data'
            END as category
         FROM INFORMATION_SCHEMA.TABLES 
         WHERE TABLE_SCHEMA = '{schema_name}' 
         AND TABLE_TYPE = 'BASE TABLE'
         ORDER BY TABLE_NAME) AS tables_query"""
        
        df_tables = spark.read \
            .format("com.microsoft.sqlserver.jdbc.spark") \
            .option("url", working_connection) \
            .option("dbtable", query) \
            .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()
        
        tables_df = df_tables.toPandas()
        
        for _, row in tables_df.iterrows():
            tables_info.append({
                'table_name': row['TABLE_NAME'],
                'full_name': f"{schema_name}.{row['TABLE_NAME']}",
                'type': row['TABLE_TYPE'],
                'category': row['category'],
                'source': 'jdbc_msi'
            })
            
        print(f"   ✅ Discovered {len(tables_info)} tables via JDBC MSI")
        
    except Exception as e:
        print(f"   ❌ JDBC MSI table discovery failed: {str(e)[:100]}...")

elif connection_method == "spark_jdbc_token":
    print("? Using Spark JDBC with Token Authentication")
    try:
        # Query INFORMATION_SCHEMA using token-based JDBC
        query = f"""
        (SELECT 
            TABLE_NAME,
            TABLE_TYPE,
            CASE 
                WHEN TABLE_NAME LIKE '%Customer%' THEN 'Customer Data'
                WHEN TABLE_NAME LIKE '%Product%' THEN 'Product Catalog'
                WHEN TABLE_NAME LIKE '%SalesOrder%' THEN 'Sales Transactions'
                WHEN TABLE_NAME LIKE '%Address%' THEN 'Address Information'
                ELSE 'Reference Data'
            END as category
         FROM INFORMATION_SCHEMA.TABLES 
         WHERE TABLE_SCHEMA = '{schema_name}' 
         AND TABLE_TYPE = 'BASE TABLE'
         ORDER BY TABLE_NAME) AS tables_query"""
        
        # Get fresh token
        token = mssparkutils.credentials.getToken(audience="https://database.windows.net/")
        
        connection_properties = {
            "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver",
            "accessToken": token
        }
        
        df_tables = spark.read.jdbc(
            url=working_connection,
            table=query,
            properties=connection_properties
        )
        
        tables_df = df_tables.toPandas()
        
        for _, row in tables_df.iterrows():
            tables_info.append({
                'table_name': row['TABLE_NAME'],
                'full_name': f"{schema_name}.{row['TABLE_NAME']}",
                'type': row['TABLE_TYPE'],
                'category': row['category'],
                'source': 'jdbc_token'
            })
            
        print(f"   ✅ Discovered {len(tables_info)} tables via JDBC Token")
        
    except Exception as e:
        print(f"   ❌ JDBC Token table discovery failed: {str(e)[:100]}...")

elif connection_method == "fabric_sql_direct":
    print("💡 Using Fabric SQL Database Direct Connection")
    try:
        # Query using Fabric SQL connection name
        query = f"""
        SELECT 
            TABLE_NAME,
            TABLE_TYPE,
            CASE 
                WHEN TABLE_NAME LIKE '%Customer%' THEN 'Customer Data'
                WHEN TABLE_NAME LIKE '%Product%' THEN 'Product Catalog'
                WHEN TABLE_NAME LIKE '%SalesOrder%' THEN 'Sales Transactions'
                WHEN TABLE_NAME LIKE '%Address%' THEN 'Address Information'
                ELSE 'Reference Data'
            END as category
        FROM {connection_name}.INFORMATION_SCHEMA.TABLES 
        WHERE TABLE_SCHEMA = '{schema_name}' 
        AND TABLE_TYPE = 'BASE TABLE'
        ORDER BY TABLE_NAME
        """
        
        tables_df = spark.sql(query).toPandas()
        
        for _, row in tables_df.iterrows():
            tables_info.append({
                'table_name': row['TABLE_NAME'],
                'full_name': f"{connection_name}.{schema_name}.{row['TABLE_NAME']}",
                'type': row['TABLE_TYPE'],
                'category': row['category'],
                'source': 'fabric_sql'
            })
            
        print(f"   ✅ Discovered {len(tables_info)} tables via Fabric SQL")
        
    except Exception as e:
        print(f"   ❌ Fabric SQL table discovery failed: {str(e)[:100]}...")

elif connection_method == "existing_tables":
    print("💡 Using Existing Lakehouse Tables/Shortcuts")
    try:
        existing_tables = spark.sql("SHOW TABLES").toPandas()
        
        # Standard SalesLT table names (without schema prefix in shortcuts)
        expected_saleslt_tables = [
            'address', 'customer', 'customeraddress', 'product', 
            'productcategory', 'productdescription', 'productmodel',
            'productmodelproductdescription', 'salesorderdetail', 'salesorderheader'
        ]
        
        # Look for exact matches and partial matches
        print(f"   🔍 Scanning {len(existing_tables)} tables in lakehouse...")
        
        for _, row in existing_tables.iterrows():
            table_name = row['tableName'].lower()
            original_name = row['tableName']
            
            # Check if this is a SalesLT table (exact match or contains expected name)
            is_saleslt_table = False
            category = 'Reference Data'
            
            if table_name in expected_saleslt_tables:
                is_saleslt_table = True
            else:
                # Check for partial matches (e.g., table might have prefix/suffix)
                for expected in expected_saleslt_tables:
                    if expected in table_name or table_name in expected:
                        is_saleslt_table = True
                        break
            
            if is_saleslt_table:
                # Determine category
                if any(x in table_name for x in ['customer', 'customeraddress']):
                    category = 'Customer Data'
                elif any(x in table_name for x in ['product', 'productcategory', 'productdescription', 'productmodel']):
                    category = 'Product Catalog'
                elif any(x in table_name for x in ['salesorder', 'order']):
                    category = 'Sales Transactions'
                elif 'address' in table_name:
                    category = 'Address Information'
                
                tables_info.append({
                    'table_name': original_name,
                    'full_name': original_name,
                    'type': 'TABLE',
                    'category': category,
                    'source': 'lakehouse_shortcuts'
                })
                
                print(f"   ✅ Found SalesLT table: {original_name} ({category})")
        
        print(f"   📊 Total SalesLT tables found: {len(tables_info)}")
        
        if len(tables_info) == 0:
            print("   ⚠️  No SalesLT tables found in shortcuts")
            print("   💡 Expected tables: Address, Customer, CustomerAddress, Product,")
            print("       ProductCategory, ProductDescription, ProductModel,")
            print("       ProductModelProductDescription, SalesOrderDetail, SalesOrderHeader")
            print("\n   🔧 To fix this:")
            print("   1. Go to your lakehouse → Tables")
            print("   2. Create shortcuts from Azure SQL Database")
            print("   3. Select all SalesLT schema tables")
        
    except Exception as e:
        print(f"   ❌ Existing tables discovery failed: {str(e)[:100]}...")

elif connection_method == "mirrored_database":
    print("💡 Using Mirrored Database")
    try:
        tables_query = f"SHOW TABLES IN {connection_name}"
        all_tables = spark.sql(tables_query).toPandas()
        
        # Filter for SalesLT-like tables
        saleslt_indicators = ['customer', 'product', 'address', 'salesorder']
        
        for _, row in all_tables.iterrows():
            table_name = row['tableName'] if 'tableName' in row else row.iloc[0]
            
            if any(indicator in table_name.lower() for indicator in saleslt_indicators):
                category = 'Unknown'
                if 'customer' in table_name.lower():
                    category = 'Customer Data'
                elif 'product' in table_name.lower():
                    category = 'Product Catalog'
                elif 'salesorder' in table_name.lower():
                    category = 'Sales Transactions'
                elif 'address' in table_name.lower():
                    category = 'Address Information'
                
                tables_info.append({
                    'table_name': table_name,
                    'full_name': f"{connection_name}.{table_name}",
                    'type': 'TABLE',
                    'category': category,
                    'source': 'mirrored_db'
                })
        
        print(f"   ✅ Found {len(tables_info)} SalesLT-like tables in mirrored database")
        
    except Exception as e:
        print(f"   ❌ Mirrored database discovery failed: {str(e)[:100]}...")

# Fallback: Use standard SalesLT table list
if not tables_info:
    print("💡 Using Standard SalesLT Table List (Fallback)")
    print("   ⚠️  Authentication failed - using expected table structure")
    
    standard_saleslt_tables = [
        ('Address', 'Address Information'),
        ('Customer', 'Customer Data'),
        ('CustomerAddress', 'Customer Data'),
        ('Product', 'Product Catalog'),
        ('ProductCategory', 'Product Catalog'),
        ('ProductDescription', 'Product Catalog'),
        ('ProductModel', 'Product Catalog'),
        ('ProductModelProductDescription', 'Product Catalog'),
        ('SalesOrderDetail', 'Sales Transactions'),
        ('SalesOrderHeader', 'Sales Transactions')
    ]
    
    for table_name, category in standard_saleslt_tables:
        tables_info.append({
            'table_name': table_name,
            'full_name': f"{schema_name}.{table_name}",
            'type': 'BASE TABLE',
            'category': category,
            'source': 'standard_list'
        })
    
    print(f"   📋 Using standard list of {len(tables_info)} SalesLT tables")

# Display discovered tables
print(f"\n📊 DISCOVERED TABLES SUMMARY")
print("-" * 60)

if tables_info:
    # Group by category
    from collections import defaultdict
    by_category = defaultdict(list)
    
    for table in tables_info:
        by_category[table['category']].append(table)
    
    total_tables = len(tables_info)
    print(f"📋 Total Tables: {total_tables}")
    print(f"📋 Source: {tables_info[0]['source']}")
    print()
    
    for category, tables in by_category.items():
        print(f"📁 {category} ({len(tables)} tables):")
        for table in tables:
            print(f"   🔗 {table['table_name']}")
    
    print(f"\n✅ Table discovery completed successfully!")
    print(f"📋 Ready to export {total_tables} tables to Bronze layer")
    
else:
    print("❌ NO TABLES DISCOVERED")
    print("💡 This indicates authentication/connection issues")
    print("🔧 Consider setting up lakehouse shortcuts or Data Factory pipelines")

print("=" * 60)

## Step 8 - Table Discovery

In [ ]:
## Step 5: Authentication-Aware Export Function

import os
import json
from datetime import datetime

def export_table_to_bronze(table_info, connection_method, connection_name, working_connection=None):
    """
    Export a single table to bronze layer using the appropriate authentication method
    Based on Fabric community guidance for SQL Server connectivity
    """
    table_name = table_info['table_name']
    full_name = table_info['full_name']
    category = table_info['category']
    source = table_info['source']
    
    print(f"? Exporting {table_name} ({category})")
    print(f"   📋 Method: {connection_method}")
    print(f"   🔗 Source: {source}")
    
    try:
        # Read data based on authentication method
        if connection_method == "spark_jdbc_msi":
            print(f"   ? Using Spark JDBC with Workspace Identity")
            
            # Use JDBC with MSI authentication
            df = spark.read \
                .format("com.microsoft.sqlserver.jdbc.spark") \
                .option("url", working_connection) \
                .option("dbtable", f"SalesLT.{table_name}") \
                .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
                .load()
                
        elif connection_method == "spark_jdbc_token":
            print(f"   🔐 Using Spark JDBC with Token Authentication")
            
            # Get fresh token for this operation
            token = mssparkutils.credentials.getToken(audience="https://database.windows.net/")
            
            connection_properties = {
                "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver",
                "accessToken": token
            }
            
            df = spark.read.jdbc(
                url=working_connection,
                table=f"SalesLT.{table_name}",
                properties=connection_properties
            )
            
        elif connection_method == "fabric_sql_direct":
            print(f"   ? Using Fabric SQL Database Direct")
            
            # Use Fabric SQL connection directly
            df = spark.sql(f"SELECT * FROM {connection_name}.SalesLT.{table_name}")
            
        elif connection_method == "existing_tables":
            print(f"   🔐 Using Existing Lakehouse Tables")
            
            # Read from existing lakehouse table
            df = spark.sql(f"SELECT * FROM {table_name}")
            
        elif connection_method == "mirrored_database":
            print(f"   ? Using Mirrored Database")
            
            # Read from mirrored database
            df = spark.sql(f"SELECT * FROM {connection_name}.{table_name}")
            
        else:
            print(f"   ⚠️  Using manual data setup - creating placeholder")
            
            # Create a placeholder DataFrame for fallback
            df = spark.createDataFrame(
                [{"status": "authentication_required", "table": table_name, "timestamp": str(datetime.now())}]
            )
        
        # Get row count and basic info
        row_count = df.count()
        columns = df.columns
        
        print(f"   📊 Loaded: {row_count:,} rows, {len(columns)} columns")
        
        # Create bronze layer path using configured target
        try:
            # Use configured bronze target path from Step 3
            base_path = globals().get('BRONZE_TARGET_PATH', 'Files/bronze/saleslt')
            bronze_path = f"{base_path}/{table_name.lower()}"
        except:
            # Fallback to default path
            bronze_path = f"Files/bronze/saleslt/{table_name.lower()}"
        
        # Add metadata columns
        df_with_metadata = df \
            .withColumn("_bronze_load_date", lit(datetime.now().strftime("%Y-%m-%d"))) \
            .withColumn("_bronze_load_timestamp", lit(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))) \
            .withColumn("_source_system", lit("SalesLT")) \
            .withColumn("_source_table", lit(table_name)) \
            .withColumn("_source_schema", lit("SalesLT")) \
            .withColumn("_auth_method", lit(connection_method)) \
            .withColumn("_load_method", lit(source))
        
        # Write to bronze layer as Parquet
        print(f"   💾 Writing to: {bronze_path}")
        
        df_with_metadata.write \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .parquet(bronze_path)
        
        # Create metadata file
        metadata = {
            "table_name": table_name,
            "category": category,
            "source_system": "SalesLT",
            "row_count": row_count,
            "column_count": len(columns),
            "columns": columns,
            "load_timestamp": datetime.now().isoformat(),
            "load_date": datetime.now().strftime("%Y-%m-%d"),
            "auth_method": connection_method,
            "connection_source": source,
            "bronze_path": bronze_path,
            "format": "parquet"
        }
        
        # Write metadata as JSON
        try:
            base_path = globals().get('BRONZE_TARGET_PATH', 'Files/bronze/saleslt')
            metadata_path = f"{base_path}/_metadata/{table_name.lower()}_metadata.json"
        except:
            metadata_path = f"Files/bronze/saleslt/_metadata/{table_name.lower()}_metadata.json"
        
        # Create metadata DataFrame and save
        metadata_df = spark.createDataFrame([metadata])
        
        try:
            base_path = globals().get('BRONZE_TARGET_PATH', 'Files/bronze/saleslt')
            metadata_write_path = f"{base_path}/_metadata/{table_name.lower()}"
        except:
            metadata_write_path = f"Files/bronze/saleslt/_metadata/{table_name.lower()}"
            
        metadata_df.coalesce(1).write \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .json(metadata_write_path)
        
        print(f"   ✅ Export completed: {row_count:,} rows")
        print(f"   📁 Data: {bronze_path}")
        print(f"   📋 Metadata: {metadata_write_path}")
        
        return {
            "success": True,
            "table_name": table_name,
            "row_count": row_count,
            "column_count": len(columns),
            "bronze_path": bronze_path,
            "auth_method": connection_method
        }
        
    except Exception as e:
        error_msg = str(e)
        print(f"   ❌ Export failed: {error_msg[:100]}...")
        
        # For authentication failures, provide specific guidance
        if any(auth_keyword in error_msg.lower() for auth_keyword in ['login', 'authentication', 'permission', 'access', 'token']):
            print(f"   💡 Authentication error detected")
            print(f"   🔧 Try: Create lakehouse shortcuts or use Data Factory")
            
        return {
            "success": False,
            "table_name": table_name,
            "error": error_msg,
            "auth_method": connection_method,
            "recommendation": "Check authentication or use lakehouse shortcuts"
        }

print("?️  EXPORT FUNCTION READY")
print("=" * 60)
print("✅ Authentication-aware export function loaded")
print("✅ Supports multiple connection methods")
print("✅ Includes community-recommended approaches")
print("✅ Bronze layer with metadata tracking")
print("✅ Authentication error handling")
print("=" * 60)

## Step 9 - Export Function

In [ ]:
## Step 6: Execute Export with Authentication Handling

from pyspark.sql.functions import lit
import time

print("🚀 STARTING SALESLT TO BRONZE EXPORT")
print("=" * 60)
print(f"? Export Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🔐 Authentication Method: {auth_method}")
print(f"📋 Connection Method: {connection_method}")
print(f"🔗 Connection/Database: {connection_name}")
print(f"? Tables to Export: {len(tables_info)}")
print()

# Check if we need to handle authentication issues
if auth_method == "manual_authentication_needed":
    print("⚠️  AUTHENTICATION SETUP REQUIRED")
    print("=" * 60)
    print("💡 Based on community guidance, here are your options:")
    print()
    print("🔧 OPTION 1: Setup Lakehouse Shortcuts (Recommended)")
    print("   1. Go to your lakehouse in Fabric")
    print("   2. Click 'New shortcut' in Tables section")
    print("   3. Select 'Azure SQL Database'")
    print("   4. Use server: gaiye-sql-db.sql.fabric.microsoft.com")
    print("   5. Database: Gaiye-SQL-DB")
    print("   6. Select all SalesLT tables")
    print("   7. Re-run this notebook - it will detect shortcuts automatically")
    print()
    print("? OPTION 2: Use Data Factory")
    print("   1. Create Data Factory pipeline")
    print("   2. Use existing connections: 'FabricSql gazho' or 'gaiye-sql-db...'")
    print("   3. Create copy activities for each SalesLT table")
    print("   4. Target: Lakehouse Files/bronze/saleslt/")
    print()
    print("🔧 OPTION 3: Grant Workspace Identity Access")
    print("   1. Go to Azure Portal > SQL Server > gaiye-sql-db")
    print("   2. Add your Fabric workspace identity as SQL admin")
    print("   3. Or grant db_datareader role on Gaiye-SQL-DB")
    print("   4. Re-run this notebook")
    print()
    print("💡 After setup, re-run from Step 3 to detect working authentication")
    print("=" * 60)

# Proceed with export if we have a working connection
export_results = []
successful_exports = 0
failed_exports = 0

if tables_info and auth_method != "manual_authentication_needed":
    print("💾 STARTING TABLE EXPORTS")
    print("-" * 60)
    
    start_time = time.time()
    
    # Create bronze directory structure
    try:
        # Ensure bronze directories exist
        spark.sql("CREATE DATABASE IF NOT EXISTS bronze")
        print("✅ Bronze database structure ready")
    except:
        pass
    
    # Export each table
    for i, table_info in enumerate(tables_info, 1):
        print(f"\n[{i}/{len(tables_info)}] {table_info['table_name']}")
        print("-" * 40)
        
        # Pass the working_connection variable for JDBC methods
        connection_to_pass = working_connection if 'working_connection' in locals() else connection_name
        
        result = export_table_to_bronze(
            table_info, 
            connection_method, 
            connection_name,
            connection_to_pass
        )
        
        export_results.append(result)
        
        if result['success']:
            successful_exports += 1
        else:
            failed_exports += 1
            
        # Small delay to avoid overwhelming the connection
        time.sleep(0.5)
    
    end_time = time.time()
    duration = end_time - start_time
    
    print(f"\n🎉 EXPORT SUMMARY")
    print("=" * 60)
    print(f"⏱️  Total Duration: {duration:.1f} seconds")
    print(f"✅ Successful Exports: {successful_exports}")
    print(f"❌ Failed Exports: {failed_exports}")
    print(f"📊 Total Tables: {len(tables_info)}")
    
    if successful_exports > 0:
        print(f"\n? BRONZE LAYER STRUCTURE:")
        print(f"   Files/bronze/saleslt/")
        
        # Show exported tables by category
        from collections import defaultdict
        by_category = defaultdict(list)
        
        for result in export_results:
            if result['success']:
                # Find table info for this result
                table_info = next((t for t in tables_info if t['table_name'] == result['table_name']), None)
                if table_info:
                    by_category[table_info['category']].append(result)
        
        for category, results in by_category.items():
            print(f"   📂 {category}:")
            for result in results:
                print(f"      📄 {result['table_name'].lower()}/ ({result['row_count']:,} rows)")
        
        print(f"\n📋 METADATA LOCATION:")
        print(f"   Files/bronze/saleslt/_metadata/")
        
        print(f"\n? NEXT STEPS:")
        print(f"   1. 🔍 Explore data: Use notebook or SQL endpoint")
        print(f"   2. 📊 Create Silver layer: Clean and transform data")
        print(f"   3. 🔗 Build relationships: Join tables for analysis")
        print(f"   4. 📈 Create reports: Use Power BI or Fabric reports")
    
    if failed_exports > 0:
        print(f"\n⚠️  EXPORT ISSUES:")
        for result in export_results:
            if not result['success']:
                print(f"   ❌ {result['table_name']}: {result.get('error', 'Unknown error')[:80]}...")
        
        print(f"\n🔧 TROUBLESHOOTING:")
        print(f"   - Check workspace identity permissions on SQL Server")
        print(f"   - Consider using lakehouse shortcuts instead")
        print(f"   - Use Data Factory with existing connections")

elif tables_info and auth_method == "manual_authentication_needed":
    print("⚠️  EXPORT BLOCKED - AUTHENTICATION REQUIRED")
    print("📋 Tables identified but authentication failed")
    print("🔧 Complete authentication setup first, then re-run")

else:
    print("❌ NO TABLES TO EXPORT")
    print("🔧 Check connection and authentication setup")

print(f"\n{'='*60}")
print(f"📋 Export process completed at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Final guidance based on authentication method
if auth_method == "workspace_identity":
    print(f"? SUCCESS: Workspace Identity authentication working!")
    print(f"🎯 This is the recommended production approach")
elif auth_method == "token_based":
    print(f"⚠️  TOKEN-BASED: Monitor for token expiration issues")
    print(f"💡 Consider switching to Workspace Identity for reliability")
elif auth_method == "fabric_native":
    print(f"💡 FABRIC SQL: Direct connection established")
elif auth_method == "manual_authentication_needed":
    print(f"🔧 SETUP REQUIRED: Follow authentication guidance above")

print(f"{'='*60}")

## Step 10 - Execute Export

In [ ]:
## Step 7: Troubleshooting & Next Steps

print("? TROUBLESHOOTING & NEXT STEPS")
print("=" * 60)
print("📋 Based on Fabric community guidance and authentication results")
print()

# Check if export was successful
export_success = 'successful_exports' in locals() and successful_exports > 0
auth_issues = auth_method == "manual_authentication_needed"

if export_success:
    print("🎉 EXPORT SUCCESSFUL!")
    print("-" * 30)
    print("✅ SalesLT data successfully exported to Bronze layer")
    print("✅ Metadata tracking in place")
    print("✅ Authentication method working")
    print()
    
    print("📊 VERIFY YOUR DATA:")
    print("   # List exported tables")
    print("   spark.sql('SHOW TABLES').show()")
    print()
    print("   # Check a sample table")
    print("   spark.sql('SELECT * FROM bronze.customer LIMIT 10').show()")
    print()
    print("   # Check row counts")
    print("   spark.sql('SELECT COUNT(*) FROM bronze.customer').show()")
    print()
    
    print("🔄 NEXT STEPS:")
    print("1. 🥈 Create Silver Layer:")
    print("   - Clean and standardize data")
    print("   - Create business-friendly views")
    print("   - Implement data quality checks")
    print()
    print("2. 🥇 Create Gold Layer:")
    print("   - Build aggregated tables")
    print("   - Create star schema for analytics")
    print("   - Optimize for reporting")
    print()
    print("3. 📊 Build Analytics:")
    print("   - Connect Power BI to Gold layer")
    print("   - Create semantic models")
    print("   - Build dashboards and reports")

elif auth_issues:
    print("🔐 AUTHENTICATION SETUP NEEDED")
    print("-" * 30)
    print("⚠️  Based on community forum insights, authentication with Azure SQL")
    print("   from Fabric notebooks has known limitations")
    print()
    
    print("🔧 RECOMMENDED SOLUTIONS (in order of preference):")
    print()
    print("1. 🎯 LAKEHOUSE SHORTCUTS (Easiest)")
    print("   ✅ No authentication issues")
    print("   ✅ Data appears as native lakehouse tables")
    print("   ✅ Automatic refresh capabilities")
    print()
    print("   Steps:")
    print("   a) Go to your lakehouse > Tables section")
    print("   b) Click 'New shortcut' > 'Azure SQL Database'")
    print("   c) Server: gaiye-sql-db.sql.fabric.microsoft.com")
    print("   d) Database: Gaiye-SQL-DB")
    print("   e) Select schema: SalesLT")
    print("   f) Select all tables or specific ones")
    print("   g) Re-run this notebook - it will detect them automatically")
    print()
    
    print("2. 🔄 DATA FACTORY PIPELINES")
    print("   ✅ Your existing connections work in Data Factory")
    print("   ✅ Scheduled refresh capabilities")
    print("   ✅ Full ETL/ELT capabilities")
    print()
    print("   Steps:")
    print("   a) Create new Data Factory")
    print("   b) Use connection: 'gaiye-sql-db.sql.fabric.microsoft;SalesLT gazho'")
    print("   c) Create copy activities for each table")
    print("   d) Target: Lakehouse Files/bronze/saleslt/")
    print("   e) Schedule for regular updates")
    print()
    
    print("3. 🔑 WORKSPACE IDENTITY SETUP")
    print("   ⚠️  Requires Azure admin permissions")
    print("   ⚠️  Community reports mixed success")
    print()
    print("   Steps:")
    print("   a) Get your Fabric workspace identity name")
    print("   b) In Azure Portal: SQL Server > Security > Entra ID admin")
    print("   c) Add workspace identity as admin or db_datareader")
    print("   d) Test connection in this notebook")
    print()
    
    print("📋 COMMUNITY INSIGHTS:")
    print("   - Workspace Identity more reliable than Managed Identity")
    print("   - Token-based auth often returns user tokens vs workspace tokens")
    print("   - JDBC connector issues common in Fabric notebooks")
    print("   - Data Factory works better for SQL Server connections")

else:
    print("❓ MIXED RESULTS")
    print("-" * 30)
    print("Some tables may have exported successfully")
    print("Check the export summary above for details")

print()
print("🔗 HELPFUL RESOURCES:")
print("   📚 Fabric Documentation: https://docs.microsoft.com/fabric")
print("   🏠 Community Forum: https://community.fabric.microsoft.com")
print("   📖 Lakehouse Shortcuts: Search 'Create shortcuts to Azure SQL'")
print("   🏭 Data Factory: Search 'Copy data from Azure SQL to Lakehouse'")

print()
print("💡 PRO TIPS:")
print("   - Lakehouse shortcuts are often easier than notebook connections")
print("   - Data Factory pipelines handle authentication better")
print("   - Test with small tables first")
print("   - Monitor workspace identity permissions carefully")
print("   - Consider hybrid approach: shortcuts + notebooks for processing")

print("\n" + "=" * 60)
print("📋 Notebook completed successfully!")
print("? Choose your preferred authentication method and proceed")
print("=" * 60)

## Step 11 - Validation

In [ ]:
# EXPLICIT TARGET LAKEHOUSE VERIFICATION
print("🔍 EXPLICIT BRONZE LAKEHOUSE VERIFICATION")
print("=" * 60)

TARGET_LAKEHOUSE = "RDS_Fabric_Foundry_workspace_Gaiye_Retail_Solution_Test_IDM_LH_bronze"
print(f"🎯 Target Lakehouse: {TARGET_LAKEHOUSE}")

# Get the configured bronze path
bronze_base = globals().get('BRONZE_TARGET_PATH', 'Files/bronze/saleslt')
print(f"📁 Configured Bronze Path: {bronze_base}")

# Method 1: Check via mount points
print(f"\n🔍 METHOD 1: Mount Point Analysis")
try:
    mounts = dbutils.fs.mounts()
    print(f"📂 All available mounts:")
    
    target_found = False
    for mount in mounts:
        mount_point = mount.mountPoint
        source = mount.source
        print(f"   🔗 {mount_point} → {source}")
        
        if TARGET_LAKEHOUSE.lower() in source.lower():
            target_found = True
            print(f"   ✅ TARGET FOUND: {mount_point}")
            
            # Check if bronze data exists in this mount
            try:
                bronze_check = f"{mount_point}bronze/saleslt"
                files = dbutils.fs.ls(bronze_check)
                print(f"   📊 Bronze data found: {len(files)} items in {bronze_check}")
                for file in files[:5]:  # Show first 5
                    print(f"      📄 {file.name}")
                if len(files) > 5:
                    print(f"      ... and {len(files)-5} more")
            except Exception as e:
                print(f"   ⚠️ No bronze data yet in {bronze_check}: {str(e)[:50]}...")
    
    if not target_found:
        print(f"   ❌ Target lakehouse not found in mounts")
        
except Exception as e:
    print(f"❌ Mount analysis failed: {str(e)}")

# Method 2: Check current lakehouse Files directory
print(f"\n🔍 METHOD 2: Current Lakehouse Files Check")
try:
    files_content = dbutils.fs.ls("Files/")
    print(f"📁 Current lakehouse Files/ contains:")
    for item in files_content:
        print(f"   📂 {item.name}")
        
        if item.name.rstrip('/') == 'bronze':
            print(f"   🎯 Bronze folder found! Checking contents...")
            try:
                bronze_content = dbutils.fs.ls("Files/bronze/")
                for bronze_item in bronze_content:
                    print(f"      📂 {bronze_item.name}")
                    
                    if bronze_item.name.rstrip('/') == 'saleslt':
                        print(f"      🎉 SalesLT folder found! Checking tables...")
                        try:
                            saleslt_content = dbutils.fs.ls("Files/bronze/saleslt/")
                            print(f"      📊 Found {len(saleslt_content)} items:")
                            for table_item in saleslt_content:
                                print(f"         📄 {table_item.name}")
                        except Exception as e:
                            print(f"      ⚠️ Could not read saleslt: {str(e)[:50]}...")
            except Exception as e:
                print(f"   ⚠️ Could not read bronze: {str(e)[:50]}...")
                
except Exception as e:
    print(f"❌ Files check failed: {str(e)}")

# Method 3: Direct path check using configured path
print(f"\n🔍 METHOD 3: Configured Path Direct Check")
try:
    # Check the exact path we're supposed to write to
    check_path = bronze_base
    print(f"📁 Checking configured path: {check_path}")
    
    content = dbutils.fs.ls(check_path)
    print(f"✅ Path accessible! Contains {len(content)} items:")
    for item in content[:10]:  # Show first 10
        print(f"   📄 {item.name}")
        
        # If it's a directory, show row count
        if item.isDir() and not item.name.startswith('_'):
            table_name = item.name.rstrip('/')
            try:
                table_path = f"{check_path}/{table_name}"
                df = spark.read.parquet(table_path)
                row_count = df.count()
                print(f"      📊 {row_count:,} rows")
            except Exception as e:
                print(f"      ⚠️ Could not read: {str(e)[:30]}...")
    
    if len(content) > 10:
        print(f"   ... and {len(content)-10} more items")
        
except Exception as e:
    print(f"❌ Configured path check failed: {str(e)}")
    print(f"💡 This suggests data was not written to the intended location")

# Method 4: Workspace context check
print(f"\n🔍 METHOD 4: Workspace Context Analysis")
try:
    # Check current database/lakehouse context
    current_db = spark.sql("SELECT current_database()").collect()[0][0]
    print(f"📊 Current database context: {current_db}")
    
    # Check if we're in the right workspace
    if TARGET_LAKEHOUSE.lower() in current_db.lower():
        print(f"✅ We appear to be in the target lakehouse context!")
    else:
        print(f"⚠️ Current context may not be the target lakehouse")
        print(f"💡 Expected: {TARGET_LAKEHOUSE}")
        print(f"💡 Current: {current_db}")

except Exception as e:
    print(f"❌ Context check failed: {str(e)}")

print(f"\n{'='*60}")
print(f"🎯 SUMMARY:")
print(f"   📋 Target: {TARGET_LAKEHOUSE}")
print(f"   📁 Path: {bronze_base}")
print(f"   🔍 Use the above methods to locate your exported data")
print(f"   💡 If data not found, re-run Step 3 configuration and Step 10 export")
print(f"{'='*60}")

In [ ]:
# Validation: Verify files exist in bronze layer
print("🔍 VALIDATION: Checking bronze layer contents...")
print("=" * 50)

bronze_base_path = "Files/bronze"

try:
    # List bronze layer directories
    if os.path.exists(bronze_base_path):
        bronze_contents = os.listdir(bronze_base_path)
        
        print(f"📁 Bronze layer contains {len(bronze_contents)} items:")
        for item in sorted(bronze_contents):
            item_path = os.path.join(bronze_base_path, item)
            if os.path.isdir(item_path):
                # Check for parquet files in the directory
                files = [f for f in os.listdir(item_path) if f.endswith('.parquet')]
                print(f"   📂 {item}/ ({len(files)} parquet files)")
            else:
                print(f"   📄 {item}")
    else:
        print(f"❌ Bronze layer path not found: {bronze_base_path}")
        
except Exception as e:
    print(f"❌ Error validating bronze layer: {str(e)}")

print("\n✅ Validation completed")

In [ ]:
# FABRIC LAKEHOUSE VALIDATION - Check bronze layer in Fabric
print("🔍 FABRIC LAKEHOUSE VALIDATION")
print("=" * 50)

try:
    # Check if bronze layer exists in Fabric lakehouse
    bronze_tables = spark.sql("SHOW TABLES").filter("tableName LIKE '*bronze*' OR tableName LIKE '*saleslt*'").toPandas()
    
    if len(bronze_tables) > 0:
        print(f"✅ Found {len(bronze_tables)} bronze/saleslt related tables:")
        for _, row in bronze_tables.iterrows():
            print(f"   📊 {row['tableName']}")
    
    # Check Files directory using Fabric file system
    try:
        # Try to list bronze directory using dbutils (Fabric file system)
        files_list = dbutils.fs.ls("Files/bronze/saleslt/")
        print(f"\n✅ Bronze layer found with {len(files_list)} items:")
        for file_info in files_list:
            print(f"   📂 {file_info.name}")
            
    except Exception as fs_error:
        # Alternative: Check using mssparkutils
        try:
            from notebookutils import mssparkutils
            files_list = mssparkutils.fs.ls("Files/bronze/saleslt/")
            print(f"\n✅ Bronze layer found with {len(files_list)} items:")
            for file_info in files_list:
                print(f"   📂 {file_info.name}")
        except:
            print(f"\n⚠️ Could not access Files/bronze/saleslt/ via file system")
            print(f"   💡 Check manually in Fabric lakehouse Files section")
    
    # Quick data sample if customer table exists
    try:
        customer_df = spark.read.parquet("Files/bronze/saleslt/customer")
        customer_count = customer_df.count()
        print(f"\n📊 SAMPLE DATA CHECK:")
        print(f"   ✅ Customer table: {customer_count:,} rows")
        
        # Show first few rows
        print(f"   📋 Sample columns: {', '.join(customer_df.columns[:5])}")
        
    except Exception as e:
        print(f"\n📊 SAMPLE DATA CHECK:")
        print(f"   ⚠️ Could not read customer data: {str(e)[:100]}...")
        
except Exception as e:
    print(f"❌ Validation error: {str(e)}")

print("\n🎯 TO VERIFY SUCCESS:")
print("   1. Go to your Fabric lakehouse")
print("   2. Check Files > bronze > saleslt > [table folders]")
print("   3. Each folder should contain .parquet files")
print("=" * 50)

## Next Steps

After running this notebook, you should have:

1. **Exported Data**: All SalesLT tables saved as Parquet files in the bronze layer
2. **Metadata**: Each file includes source tracking and extraction timestamps
3. **Summary Report**: JSON file with complete export details
4. **File Organization**: Each table in its own folder within `/Files/bronze/`

### Recommended Next Actions:

- **Review the data**: Check the bronze layer files to ensure data quality
- **Create silver layer transformations**: Build data pipelines to clean and standardize the data
- **Map to retail model**: Align the SalesLT schema with your retail data model requirements
- **Set up monitoring**: Create alerts for data freshness and quality

### Troubleshooting:

- **Connection issues**: Verify your Fabric workspace has access to the SQL database
- **Permission errors**: Ensure you have read access to SalesLT schema and write access to the lakehouse
- **Large tables**: For very large tables, consider implementing chunked processing
- **Data types**: Some SQL Server data types may need special handling during export

## Data Factory Pipeline Alternative

If you prefer using **Data Factory** instead of this notebook, here's how to create a dynamic pipeline:

### 1. Create Parameterized Pipeline

Create a Data Factory pipeline with these components:

**Pipeline Parameters:**
- `sourceSchema` = "SalesLT"
- `targetPath` = "Files/bronze/saleslt"

**Activities:**
1. **Lookup Activity**: Get table list from `INFORMATION_SCHEMA.TABLES`
2. **ForEach Activity**: Loop through discovered tables  
3. **Copy Data Activity**: Copy each table dynamically

### 2. Connection Configuration

**Source (Fabric SQL Database):**
- **Server**: `Gaiye-SQL-DB.sql.fabric.microsoft.com`
- **Database**: `Gaiye-SQL-DB`
- **Authentication**: `Organizational Account`

**Sink (Lakehouse):**
- **Path**: `@concat(pipeline().parameters.targetPath, '/', toLower(item().TABLE_NAME), '/', toLower(item().TABLE_NAME), '.parquet')`
- **Format**: `Parquet`

### 3. Dynamic Query

**Lookup Query:**
```sql
SELECT TABLE_NAME 
FROM INFORMATION_SCHEMA.TABLES 
WHERE TABLE_SCHEMA = '@{pipeline().parameters.sourceSchema}' 
AND TABLE_TYPE = 'BASE TABLE'
```

**Copy Data Source Query:**
```sql
SELECT * FROM [@{pipeline().parameters.sourceSchema}].[@{item().TABLE_NAME}]
```

This approach gives you the same dynamic table discovery but through Data Factory's visual interface.